In [ ]:
#importing required libraries and warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')

**Importing CSV File**

In [ ]:

application_data = pd.read_csv('application_data.csv')

#https://drive.google.com/drive/folders/16RQztUqCfJOlbooHqYlJrp6Q7iL65uZB

***DATA LOADING***

In [ ]:
application_data.head()

In [ ]:
application_data.shape

**Identification of Missing Values**

In [ ]:

application_data.info()


**Dropping the columns with % of null values more than 30%**


In [ ]:
100*application_data.isnull().mean()

In [ ]:
cut_off=30
cols_to_drop=list(application_data.columns[100*application_data.isnull().mean()>cut_off])
print(cols_to_drop,len(cols_to_drop))

In [ ]:
df1=application_data.drop(cols_to_drop,axis=1)

In [ ]:
df1.shape

In [ ]:
df1.info()

***Listing and Deleting the columns which are not much relavent for analysis***

In [ ]:
cols_irrelevant = ['DAYS_REGISTRATION','FLAG_MOBIL','FLAG_EMP_PHONE','FLAG_WORK_PHONE','FLAG_CONT_MOBILE',
                   'FLAG_PHONE','FLAG_EMAIL','WEEKDAY_APPR_PROCESS_START','HOUR_APPR_PROCESS_START','LIVE_REGION_NOT_WORK_REGION',
                   'REG_CITY_NOT_LIVE_CITY','REG_CITY_NOT_WORK_CITY','LIVE_CITY_NOT_WORK_CITY','DAYS_LAST_PHONE_CHANGE',
                  'OBS_30_CNT_SOCIAL_CIRCLE','DEF_30_CNT_SOCIAL_CIRCLE','OBS_60_CNT_SOCIAL_CIRCLE','DEF_60_CNT_SOCIAL_CIRCLE',
                  'NAME_TYPE_SUITE']
cols_irrelevant

In [ ]:
df1 = df1.drop(cols_irrelevant,axis=1)

In [ ]:
df1.shape

In [ ]:
df1.info()

**Dividing the columns into Categorical and Numerical**

In [ ]:
cat_cols=list(df1.dtypes[df1.dtypes=='object'].index)

In [ ]:
num_cols=list(df1.dtypes[df1.dtypes!='object'].index)

In [ ]:
df1[cat_cols].head()

In [ ]:
df1[num_cols].head()

In [ ]:
df1[cat_cols].shape

In [ ]:
df1[num_cols].shape

**Imputation of Data** 


**Handling data fo Numerical Columns**

In [ ]:
100*df1[num_cols].isnull().mean()

In [ ]:
#Handling data for EXT_SOURCE_2 and EXT_SOURCE_3 column

In [ ]:
(df1['EXT_SOURCE_2'].isnull() & df1['EXT_SOURCE_3'].isnull()).sum()

In [ ]:
#As count of empty rows in both rows are low , deleting the rows from total of 307511 is prefered
df1=df1[~(df1['EXT_SOURCE_2'].isnull() & df1['EXT_SOURCE_3'].isnull())]

In [ ]:
round(df1[['EXT_SOURCE_2','EXT_SOURCE_3']].describe())

In [ ]:
# As mean and median values are almost same for both EXT_SOURCE_2,EXT_SOURCE_3, filling  mean to the null values 
cols=df1[['EXT_SOURCE_2','EXT_SOURCE_3']]
for col in cols:
    mean=df1[col].mean()
    df1[col]=df1[col].fillna(mean)


In [ ]:
df1[num_cols].isnull().mean()

In [ ]:
#Handling missing rows for AMT_REQ_CREDIT_BUREAU columns
round(df1[['AMT_REQ_CREDIT_BUREAU_HOUR','AMT_REQ_CREDIT_BUREAU_DAY','AMT_REQ_CREDIT_BUREAU_WEEK','AMT_REQ_CREDIT_BUREAU_MON','AMT_REQ_CREDIT_BUREAU_QRT','AMT_REQ_CREDIT_BUREAU_YEAR']].describe())

In [ ]:
#Since the standard deviation is high , Its not advisable to fill NA with mean or median. Hence checking for most common value 
df1[['AMT_REQ_CREDIT_BUREAU_HOUR','AMT_REQ_CREDIT_BUREAU_DAY','AMT_REQ_CREDIT_BUREAU_WEEK','AMT_REQ_CREDIT_BUREAU_MON','AMT_REQ_CREDIT_BUREAU_QRT','AMT_REQ_CREDIT_BUREAU_YEAR']].value_counts()

In [ ]:
#As 0 is the most common value for all the columns replacing Null values with 0
cols=df1[['AMT_REQ_CREDIT_BUREAU_HOUR','AMT_REQ_CREDIT_BUREAU_DAY','AMT_REQ_CREDIT_BUREAU_WEEK','AMT_REQ_CREDIT_BUREAU_MON','AMT_REQ_CREDIT_BUREAU_QRT','AMT_REQ_CREDIT_BUREAU_YEAR']]
for col in cols:
    df1[col]=df1[col].fillna(0)

In [ ]:
100*df1[num_cols].isnull().mean()

***Handling missing rows in AMT_ANNUITY,AMT_GOODS_PRICE ,CNT_FAM_MEMBERS columns***

In [ ]:
#As the % of missing values is less than 0.1% in all the three columns replacing null values with the median value of the respective column
cols=df1[['AMT_ANNUITY','AMT_GOODS_PRICE','CNT_FAM_MEMBERS']]
for col in cols:
    mean=df1[col].mean()
    df1[col]=df1[col].fillna(mean)

In [ ]:
100*df1[num_cols].isnull().mean()

**Handling Data for Categorical Columns**

***Replacing missing values with mode in case of categorical columns***

In [ ]:
100*df1[cat_cols].isnull().mean()

In [ ]:
for col in cat_cols:
    mod=df1[col].mode()[0] 
    df1[col]=df1[col].fillna(mod)

In [ ]:
df1[cat_cols].isnull().mean()

We have a dataframe with no null values. 
Now converting the DAYS_BIRTH and DAYS_EMPLOYED to AGE and WORK_EXPERIENCE columns for better readability and analysis.


In [ ]:
#Creating age column
df1['AGE'] = abs(df1['DAYS_BIRTH']//365)

In [ ]:
#Creating Work_experience column
df1['Work_Experience'] = abs(df1['DAYS_EMPLOYED']//365)

In [ ]:
#Dropping DAYS_BIRTH and DAYS_EMPLOYED columns 
df1 = df1.drop(['DAYS_BIRTH','DAYS_EMPLOYED'],axis=1)

**To analyse the category of income,adding column income group with low,medium and high depending on the AMT_INCOME_TOTAL column**


In [ ]:
round(df1['AMT_INCOME_TOTAL'].describe())

In [ ]:
    df1['Income_Group']= pd.cut(df1.AMT_INCOME_TOTAL,[0,130000,202500,118000000],labels=['Low','Medium','High'])
#dividing incomes based on the IQR values above

In [ ]:
df1[['AMT_INCOME_TOTAL','Income_Group']].head(20)

**To analyse the AGE group,adding column Age_Group depending on AGE  column**

In [ ]:
df1['Age_Group']=pd.cut(df1.AGE,[0,30,60,1000],labels=['Young','Middle-Aged','Old-Aged'])

In [ ]:
df1[['AGE','Age_Group']].head(20)

In [ ]:
df1.TARGET.value_counts(normalize=True).plot.bar()
plt.show()

**To EASE Analysis , Dividing data into two i.e Defaulters and Non Defaulters based on the Target column. 1-Defaulters , 0-non Defaulters**

In [ ]:
Defaulters = df1[df1['TARGET']==1]

In [ ]:
Defaulters.head()

In [ ]:
#Dividing into categorical and numerical columns
Dcat_cols=list(Defaulters.dtypes[df1.dtypes=='object'].index)

In [ ]:
Dnum_cols=list(Defaulters.dtypes[df1.dtypes!='object'].index)

In [ ]:
Non_Defaulters= df1[df1['TARGET']==0]

In [ ]:
Non_Defaulters.head()

In [ ]:
#Dividing into Categorical and numerical columns
Ncat_cols=list(Non_Defaulters.dtypes[df1.dtypes=='object'].index)

In [ ]:
Nnum_cols=list(Non_Defaulters.dtypes[df1.dtypes!='object'].index)

**EDA**

**Univariate Analysis for Defaulters**

***Using histogram for analysis of  numerical columns***

In [ ]:
for col in Dnum_cols:
    sns.histplot(x=df1[col])
    plt.show()

**As there is very minimum information available from the below columns, Dropping them for future analysis**

In [ ]:
cols_to_drop= Defaulters[['SK_ID_CURR','TARGET','AMT_INCOME_TOTAL','REG_REGION_NOT_LIVE_REGION','REG_REGION_NOT_WORK_REGION','FLAG_DOCUMENT_2','FLAG_DOCUMENT_4','FLAG_DOCUMENT_5','FLAG_DOCUMENT_6','FLAG_DOCUMENT_7','FLAG_DOCUMENT_8','FLAG_DOCUMENT_9','FLAG_DOCUMENT_10','FLAG_DOCUMENT_11','FLAG_DOCUMENT_12','FLAG_DOCUMENT_13','FLAG_DOCUMENT_14','FLAG_DOCUMENT_15','FLAG_DOCUMENT_16','FLAG_DOCUMENT_17','FLAG_DOCUMENT_18','FLAG_DOCUMENT_19','FLAG_DOCUMENT_20','FLAG_DOCUMENT_21','AMT_REQ_CREDIT_BUREAU_HOUR','AMT_REQ_CREDIT_BUREAU_DAY','AMT_REQ_CREDIT_BUREAU_WEEK','AMT_REQ_CREDIT_BUREAU_MON','AMT_REQ_CREDIT_BUREAU_QRT']]
Defaulters= Defaulters.drop(cols_to_drop,axis=1)

In [ ]:
Defaulters.shape

***Using count plot for categorical columns***

In [ ]:
for col in Dcat_cols:
    sns.countplot(x=df1[col])
    plt.xticks(rotation=90)
    plt.show()

**Univariate Analysis for Non-Defaulters**

In [ ]:
for col in Nnum_cols:
    sns.histplot(x=df1[col])
    plt.show()

**As there is very minimum information available from the below columns, Dropping them for future analysis**

In [ ]:
cols_to_drop= Non_Defaulters[['SK_ID_CURR','TARGET','AMT_INCOME_TOTAL','REG_REGION_NOT_LIVE_REGION','REG_REGION_NOT_WORK_REGION','FLAG_DOCUMENT_2','FLAG_DOCUMENT_4','FLAG_DOCUMENT_5','FLAG_DOCUMENT_6','FLAG_DOCUMENT_7','FLAG_DOCUMENT_8','FLAG_DOCUMENT_9','FLAG_DOCUMENT_10','FLAG_DOCUMENT_11','FLAG_DOCUMENT_12','FLAG_DOCUMENT_13','FLAG_DOCUMENT_14','FLAG_DOCUMENT_15','FLAG_DOCUMENT_16','FLAG_DOCUMENT_17','FLAG_DOCUMENT_18','FLAG_DOCUMENT_19','FLAG_DOCUMENT_20','FLAG_DOCUMENT_21','AMT_REQ_CREDIT_BUREAU_HOUR','AMT_REQ_CREDIT_BUREAU_DAY','AMT_REQ_CREDIT_BUREAU_WEEK','AMT_REQ_CREDIT_BUREAU_MON','AMT_REQ_CREDIT_BUREAU_QRT']]
Non_Defaulters= Non_Defaulters.drop(cols_to_drop,axis=1)



In [ ]:
Non_Defaulters.shape

***Using count plot for categorical columns***

In [ ]:
for col in Ncat_cols:
    sns.countplot(x=df1[col])
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
Defaulters.info()

***Comparing Defaulters and Non-Defaulters for below columns using sub-plots***

**Defaulters and Non Defaulters based on loan type**

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
sns.countplot(ax=axes[0], data=Defaulters, x='NAME_CONTRACT_TYPE')
axes[0].set_title('Defaulters')
axes[0].set_xlabel('Contract Type')
axes[0].set_ylabel('Count')
sns.countplot(ax=axes[1], data=Non_Defaulters, x='NAME_CONTRACT_TYPE')
axes[1].set_title('Non-Defaulters')
axes[1].set_xlabel('Contract Type')
axes[1].set_ylabel('Count')
plt.tight_layout()
plt.show()

Observation
--> We can see for both defaulters and non defaulters the  majority loan type is Cash loan

**Defaulters and Non Defaulters based on gender type**

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
sns.countplot(ax=axes[0], data=Defaulters, x='CODE_GENDER')
axes[0].set_title('Defaulters')
axes[0].set_xlabel('Gender')
axes[0].set_ylabel('Count')
sns.countplot(ax=axes[1], data=Non_Defaulters, x='CODE_GENDER')
axes[1].set_title('Non-Defaulters')
axes[1].set_xlabel('Gender')
axes[1].set_ylabel('Count')
plt.tight_layout()
plt.show()


Observation-->Count of Females is slightly  more for Defaulters where as count of females who is non defaulters are twice as that of males


**Defaulters and Non Defaulters based on credit amount**

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
sns.distplot(Defaulters['AMT_CREDIT'], ax=axes[0])
axes[0].set_title('Defaulters')
axes[0].set_xlabel('Credit Amount')
axes[0].set_ylabel('Count')
sns.distplot(Non_Defaulters['AMT_CREDIT'], ax=axes[1])
axes[1].set_title('Non-Defaulters')
axes[1].set_xlabel('Credit Amount')
axes[1].set_ylabel('Count')
plt.tight_layout()
plt.show()

Observation --> As the loan amount increases defaulters and non defaulters have increased till a pint but there is gradual decrease after that with increase of credit amount 

 **Defaulters and Non Defaulters based on income type**

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
sns.countplot(ax=axes[0], data=Defaulters, x='NAME_INCOME_TYPE')
axes[0].set_title('Defaulters')
axes[0].set_xlabel('Income Type')
axes[0].set_ylabel('Count')
axes[0].tick_params(axis='x', rotation=90)
sns.countplot(ax=axes[1], data=Non_Defaulters, x='NAME_INCOME_TYPE')
axes[1].set_title('Non-Defaulters')
axes[1].set_xlabel('Income Type')
axes[1].set_ylabel('Count')
axes[1].tick_params(axis='x', rotation=90)
plt.tight_layout()
plt.show()

Observation--> Working Professionals are the one to take more loans and also the leading one in both Defaulters and Non Defaulters 

**Defaulters and Non Defaulters based on Education**

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
sns.countplot(ax=axes[0], data=Defaulters, x='NAME_EDUCATION_TYPE')
axes[0].set_title('Defaulters')
axes[0].set_xlabel('Education Type')
axes[0].set_ylabel('Count')
axes[0].tick_params(axis='x', rotation=90)
sns.countplot(ax=axes[1], data=Non_Defaulters, x='NAME_EDUCATION_TYPE')
axes[1].set_title('Non-Defaulters')
axes[1].set_xlabel('Education Type')
axes[1].set_ylabel('Count')
axes[1].tick_params(axis='x', rotation=90)
plt.tight_layout()
plt.show()

Observation--> Clients with Secondary/secondary special  are the one to take more loans and also the leading one in both Defaulters and Non Defaulters

**Defaulters and Non Defaulters based on Work Experience**

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
sns.countplot(ax=axes[0], data=Defaulters, x='Work_Experience')
axes[0].set_title('Defaulters')
axes[0].set_xlabel('Work Experience')
axes[0].set_ylabel('Count')
axes[0].tick_params(axis='x', rotation=90)
sns.countplot(ax=axes[1], data=Non_Defaulters, x='Work_Experience')
axes[1].set_title('Non-Defaulters')
axes[1].set_xlabel('Work Experience')
axes[1].set_ylabel('Count')
axes[1].tick_params(axis='x', rotation=90)
plt.tight_layout()
plt.show()

Observation--> Clients with 1-5 are the one to take more loans and also the leading one in both Defaulters and Non Defaulters

**SEGMENTED UNIVARIATE ANALYSIS**

As we have already divided Age and Income into Age Group and Income Group columns , applying univariate analysis for both the columns.

**Defaulters and Non Defaulters based on Age Group**

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
sns.countplot(ax=axes[0], data=Defaulters, x='Age_Group')
axes[0].set_title('Defaulters')
axes[0].set_xlabel('Age')
axes[0].set_ylabel('Count')
sns.countplot(ax=axes[1], data=Non_Defaulters, x='Age_Group')
axes[1].set_title('Non-Defaulters')
axes[1].set_xlabel('Age')
axes[1].set_ylabel('Count')
plt.tight_layout()
plt.show()

Observation --> Clients who are middle aged(30-60yrs) are the one to take more loans and also the leading one in both Defaulters and Non Defaulters

**Defaulters and Non Defaulters based on Income Group**

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
sns.countplot(ax=axes[0], data=Defaulters, x='Income_Group')
axes[0].set_title('Defaulters')
axes[0].set_xlabel('Income')
axes[0].set_ylabel('Count')
sns.countplot(ax=axes[1], data=Non_Defaulters, x='Income_Group')
axes[1].set_title('Non-Defaulters')
axes[1].set_xlabel('Income')
axes[1].set_ylabel('Count')
plt.tight_layout()
plt.show()

Observation --> Clients who are in medium income range are the one to take more loans and also the leading one in both Defaulters and Non Defaulters slightly beating the low income range

**Key takeaways from Univariate and Segmented Univariate Analysis**

1. 91% of Clients applied for loan did not have any payment difficulties where as 9% of Clients are with payment difficulties: he/she had late payment      more than X days on at least one of the first Y installments of the loan
2. Clients are more interested in taking Cash Loans than Revolving Loans.
3. Clients who are Married,with no children are the one to take more loans than other categories.
4. Clients who are Middle aged(30-60yrs),with medium income range(130000-202500 per annum) and with 1-5 yrs of Work Experience are the one to take more  loans than other categories.
5. Clients who are Females are the one to take more loans than Males
6. Clients who own a House/Apartment are more likely to take loans than Others.


**BIVARIATE ANALYSIS**

To perform bivaariate analysis let us use the numeric and categorical columns of Defaulters and Non-Defaulters

**Bivariate Analysis of Numeric and categorical columns of Non-Defaulters**

In [ ]:
Nnum_cols=list(Non_Defaulters.dtypes[df1.dtypes!='object'].index)

**Analysis of Numerical vs Numerical columns**

In [ ]:
for col1 in Nnum_cols:
    for col2 in Nnum_cols:
        if col1!=col2:
            plt.figure(figsize=(5,5))
            sns.scatterplot(x=Non_Defaulters[col1],y=Non_Defaulters[col2])
            plt.show()

**Analysis**



1.Regional population relative with index 0.1-0.3  have take more loans than other 
2.Regional Rating and Regional rating of Client wrt city of rating 2 are  the areas with more number of loans 
3.Higher Income Group has more amount Credited than lower and middle income group
4.Clients who are of Middle Age(30-60) has taken more loans than young(<30) and Senior citizens(>60)
5.Clients with higher external source score havemore loan amount

**Analysis of Categorical vs Categorical columns**

In [ ]:
Ncat_cols = [col for col in Ncat_cols if col != 'ORGANIZATION_TYPE']

In [ ]:
for col1 in Ncat_cols:
    for col2 in Ncat_cols:
        if col1!=col2:
            plt.figure(figsize=(5,3))
            sns.boxplot(x=Non_Defaulters[col1],y=Non_Defaulters[col2])
            plt.xticks(rotation=90)
            plt.show()

**Observations**

1.Cash Loans and revolving loans are mostly spread across State Servant, Working , Commercil Associates and Pensioners. 
2.It is interesting that not cash loan but Revolving loans were opted by few  Business class and Maternity leave category.

**Analysis of Numerical vs Categorical columns**

In [ ]:
for col1 in Ncat_cols:
    for col2 in Nnum_cols:
            plt.figure(figsize=(6,6))
            sns.boxplot(x=Non_Defaulters[col1],y=Non_Defaulters[col2])
            plt.xticks(rotation=90)
            plt.show()

**Analysis**
credit 


1.There is no credit amount differnce between client owning cars or realty.
2.Revolving loans have less credit amount compared to cash loans 
3.Clients taking revolving loans are mostly middle aged compared to cash loans which is opted by all age groups
4.Thought the loans taken by Male and Female are almost in same range, the income group of Females taking loan is from Low-Medium and Males from Medium   to High 
5.Business Man tend to take more Credit Amount of loan than other classes 
6.Clients with higher education and academic degree tend to take more Credit Amount of loan
7.Married and separated people tend to take more credit amount of loans 
8.Housing type and organisation type  doesn’t seem to have affect on the loan credit 

**Bivariate Analysis of Numeric and categorical columns of Defaulters**

**Analysis of Numerical vs Numerical columns**

In [ ]:
Dnum_cols=list(Defaulters.dtypes[df1.dtypes!='object'].index)

In [ ]:
for col1 in Dnum_cols:
    for col2 in Dnum_cols:
        if col1!=col2:
            plt.figure(figsize=(5,5))
            sns.scatterplot(x=Defaulters[col1],y=Defaulters[col2])
            plt.show()

**Analysis**

Clients with Higher Income are more likely to be defaulters with payment difficulties
Clients of middle aged group are more likely to be defaulters 

**Analysis of Categorical vs Categorical columns**

In [ ]:
Dcat_cols = [col for col in Dcat_cols if col != 'ORGANIZATION_TYPE']

In [ ]:
for col1 in Dcat_cols:
    for col2 in Dcat_cols:
        if col1!=col2:
            plt.figure(figsize=(5,3))
            sns.boxplot(x=Defaulters[col1],y=Defaulters[col2])
            plt.xticks(rotation=90)
            plt.show()

**Observation**

1.Working and commercial associates and some pensioners tend to have payment difficulties for both cash and revolving loans 
2.Mostly Married clients have payment difficulties for Cash loans where as range of  Married,Widowers,Single people tend to have difficulties for Revolving loans 
3.Most of Male and Female customers opted for Cash loans have difficlties in Paying with few of them in revolving loans 


**Analysis of Numerical vs Categorical columns**

In [ ]:
for col1 in Dcat_cols:
    for col2 in Dnum_cols:
            plt.figure(figsize=(5,3))
            sns.boxplot(x=Defaulters[col1],y=Defaulters[col2])
            plt.xticks(rotation=90)
            plt.show()

**Observation**

1.There is no credit amount differnce between client owning cars or realty.
2.Revolving loans have less credit amount compared to cash loans 
3.Clients with external source score with 0.3-0.6 IQR tend to have payment difficulties
4.Clients of all income types seem to have equal distribution of  loan payment difficulties 

**Multivariate Analysis**

In [ ]:
corr_cols = ['TARGET','AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY','AMT_GOODS_PRICE','AGE','EXT_SOURCE_2','EXT_SOURCE_3','REGION_RATING_CLIENT']

In [ ]:
df_corr = df1[corr_cols]
df_corr.head()

In [ ]:
plt.figure(figsize=(8,8))
sns.heatmap(df_corr.corr(),annot=True,cmap='RdYlGn')
plt.show()

****Key takeaways from Multivariate Analysis****

1.Credit amount and Goods Price have strong Correaltion (0.99)
2.Credit Amount  and Annuity have good correlation (0.77)3.Goods Price and Annuity have good correlationE (0.77)
4.Regional Rating and External Source score have highest negative correlation(-0.29)